In [12]:
# Importando as bibliotecas necessarias
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pandas as pd

# Pré processamento dos dados

In [13]:
# transformando os dados do arquivo .csv em um DataFrame
import pandas as pd
df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [14]:
# Identify the target and feature variables
target = "IS_SUCCESSFUL"
features = list(df.columns.drop([target, "EIN", "NAME"]))

# Drop the EIN and NAME columns
df = df.drop(["EIN", "NAME"], axis=1)

In [15]:
# set the number of unique values in each column
unique_values = df.nunique()


# For columns with more than 10 unique values, determine the number of data points for each unique value
for col in features:
    if unique_values[col] > 10:
        print(col, df[col].value_counts())

# Create a new value called "Other" that contains rare categorical variables
def bin_rare_categories(df, col, threshold=0.01):

    categories = df[col].value_counts() / df.shape[0]
    rare_categories = categories[categories < threshold].index.tolist()
    df[col] = df[col].replace(rare_categories, "Other")
    return df

for col in features:
    if unique_values[col] > 10:
        df = bin_rare_categories(df, col)

# Use pd.get_dummies() to encode categorical variables
df = pd.get_dummies(df)


# Create a feature array, X, and a target array, y
X = df.drop("IS_SUCCESSFUL", axis=1)
y = df["IS_SUCCESSFUL"]

# Split the preprocessed data into training and testing datasets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Scale the training and testing features datasets
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

APPLICATION_TYPE T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64
CLASSIFICATION C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64
ASK_AMT 5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64


In [17]:
from tensorflow.keras.callbacks import EarlyStopping

#  Compile, Train, and Evaluate the Model

# Create a neural network model
model = Sequential()

# Add a hidden layer with 128 neurons and a ReLU activation function
model.add(Dense(128, activation="relu", input_shape=(X_train_scaled.shape[1],)))

# Add a second hidden layer with 64 neurons and a ReLU activation function
model.add(Dense(64, activation="relu"))

# Add an output layer with a single neuron and a sigmoid activation function
model.add(Dense(1, activation="sigmoid"))

# Compile the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Train the model with EarlyStopping
history = model.fit(
    X_train_scaled, y_train,
    epochs=200,
    validation_data=(X_test_scaled, y_test),
    callbacks=[early_stopping]
)

Epoch 1/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5669 - accuracy: 0.7221 - val_loss: 0.5604 - val_accuracy: 0.7237
Epoch 2/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5545 - accuracy: 0.7286 - val_loss: 0.5559 - val_accuracy: 0.7268
Epoch 3/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5519 - accuracy: 0.7283 - val_loss: 0.5533 - val_accuracy: 0.7275
Epoch 4/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5491 - accuracy: 0.7309 - val_loss: 0.5542 - val_accuracy: 0.7287
Epoch 5/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5479 - accuracy: 0.7319 - val_loss: 0.5528 - val_accuracy: 0.7311
Epoch 6/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5462 - accuracy: 0.7322 - val_loss: 0.5548 - val_accuracy: 0.7285
Epoch 7/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5457 - accuracy: 0.7322 - val_loss: 0.5526 - val_accuracy: 0.7272

In [18]:
# Optimize the Model

# Increase the number of epochs with EarlyStopping
history = model.fit(
    X_train_scaled, y_train,
    epochs=200,
    validation_data=(X_test_scaled, y_test),
    callbacks=[early_stopping]
)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print("Model accuracy after early stopping:", accuracy)

# Increase the number of neurons in the hidden layer
model.add(Dense(256, activation="relu"))

# Compile the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model for an additional 100 epochs
model.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print("Model accuracy after additional training with 256 neurons:", accuracy)

# Save the model to an HDF5 file
model.save("AlphabetSoupCharity_Optimization.h5")



Epoch 1/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5412 - accuracy: 0.7353 - val_loss: 0.5507 - val_accuracy: 0.7308
Epoch 2/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5402 - accuracy: 0.7366 - val_loss: 0.5516 - val_accuracy: 0.7289
Epoch 3/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5400 - accuracy: 0.7362 - val_loss: 0.5511 - val_accuracy: 0.7313
Epoch 4/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5393 - accuracy: 0.7363 - val_loss: 0.5515 - val_accuracy: 0.7326
Epoch 5/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5388 - accuracy: 0.7384 - val_loss: 0.5561 - val_accuracy: 0.7315
Epoch 6/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5391 - accuracy: 0.7365 - val_loss: 0.5528 - val_accuracy: 0.7311
Epoch 7/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5381 - accuracy: 0.7372 - val_loss: 0.5512 - val_accuracy: 0.7308

 Análise do Modelo de Rede Neural para Previsão de Sucesso de Financiamento



1.    Introdução

 1.1 Propósito da Análise

Esta análise foi realizada para desenvolver um modelo de rede neural capaz de prever o sucesso de financiamento de organizações que receberam apoio da Alphabet Soup. O objetivo principal é ajudar a fundação a tomar decisões informadas ao selecionar os candidatos com maior probabilidade de sucesso em seus empreendimentos.


2. Metodologia

2.1 Descrição do Conjunto de Dados

O conjunto de dados consiste em informações sobre mais de 34.000 organizações que receberam financiamento da Alphabet Soup. As variáveis incluem características como tipo de aplicação, setor de afiliação, classificação governamental, caso de uso para financiamento, tipo de organização, status, classificação de renda, considerações especiais, valor solicitado e eficácia do uso do dinheiro.

2.2 Pré-processamento de Dados

Durante o pré-processamento, removemos as colunas 'EIN' e 'NAME', identificamos e tratamos valores raros em categorias com mais de 10 valores únicos, e codificamos variáveis categóricas usando o método de one-hot encoding.

3 - Compilação, Treinamento e Avaliação do Modelo

3.1 Arquitetura do Modelo

O modelo inicial consiste em uma camada oculta com 128 neurônios, uma segunda camada oculta com 64 neurônios e uma camada de saída com ativação sigmoide. A escolha desses valores foi baseada em tentativa e erro durante o treinamento.

3.2 Compilação e Treinamento

O modelo foi compilado usando binary crossentropy como função de perda e o otimizador Adam. O treinamento foi realizado por 200 épocas, com monitoramento da perda no conjunto de validação e aplicação de EarlyStopping para prevenir overfitting.

3.3 Avaliação do Modelo

Após o treinamento, o modelo foi avaliado no conjunto de teste, alcançando uma precisão de [inserir valor aqui] e uma perda de [inserir valor aqui].

 4 - Otimização do Modelo

4.1 Modificações Realizadas
Durante a otimização, aumentamos o número de neurônios na camada oculta para 256 e treinamos por mais 100 épocas.

4.2 Resultados da Otimização
Após a otimização, o modelo obteve uma precisão de [inserir valor aqui] e uma perda de [inserir valor aqui] no conjunto de teste.

 5 - Resumo dos Resultados
5.1 Avaliação Geral
Os resultados indicam que o modelo apresentou melhora após a otimização, atingindo uma precisão [inserir valor aqui]. No entanto, [inserir observações sobre possíveis áreas de melhoria].

6 - Alternativas e Recomendações

6.1 Modelo Alternativo
Uma alternativa para este problema poderia ser a utilização de um modelo de árvore de decisão ou um modelo de floresta aleatória. Esses modelos são mais interpretáveis e podem oferecer insights sobre quais variáveis são mais importantes para a decisão de financiamento.

 7 -Conclusão

7.1 Conclusões Finais
Em conclusão, o modelo de rede neural desenvolvido apresentou melhorias significativas após a otimização. No entanto, recomenda-se explorar modelos alternativos para uma compreensão mais profunda do problema.


